# Data Engineering

Now that we've loaded our data, it's time to do some real data engineering to answer our original question.

At this stage, Iceberg fades into the background, but we're able to pick and choose query engines to perform the various steps - this is the true power of Iceberg

In [1]:
import sqlalchemy as sa
from utils import engine, catalog
import polars as pl
pl.Config.set_fmt_str_lengths(50)
pl.Config.set_thousands_separator(True)

polars.config.Config

We want a way of identifying a given property, so hashing the address related fields seems the easiest. We'll create a dimension table for those addresses, so we can move those rows out of our final data, without losing the ability to filter later. 
```{note} SQL Partitioning
Note that we're adding partitioning to our tables directly through SQL here using a WITH statement on the CREATE TABLE
```

In [2]:
dim_address_sql = """
CREATE OR REPLACE TABLE housing.dim_address 
    WITH ( partitioning = ARRAY['bucket(address_id, 10)'] )
    AS (
    SELECT DISTINCT to_hex(md5(cast(
        coalesce(paon, '') ||
        coalesce(saon, '') ||
        coalesce(street, '') ||
        coalesce(locality, '') ||
        coalesce(town, '') ||
        coalesce(district, '') ||
        coalesce(county, '') ||
        coalesce(postcode, '')
    as varbinary))) AS address_id,
      paon,
      saon,
      street,
      locality,
      town,
      district,
      county,
      postcode
FROM housing.staging_prices)
"""

As described in the data dictionary, the monthly files incluce a `record_status` column which indicates whether a given record is a new record or if it is deleting or updating an existing record. In moving from our staging table to our fact table, we clean our data to ensure we respect the record_status

In [3]:
fct_prices_sql = """
CREATE OR REPLACE TABLE housing.fct_house_prices
    WITH ( partitioning = ARRAY['year(date_of_transfer)'] ) AS (
        WITH ranked_records AS (
            SELECT *,
            ROW_NUMBER () OVER (PARTITION BY transaction_id ORDER BY month(date_of_transfer) DESC) AS rn
            FROM housing.staging_prices
    ),
    latest_records AS (
        SELECT *
        FROM ranked_records
        WHERE rn = 1
    ),
    with_address_id AS (
        SELECT to_hex(md5(cast (
                coalesce(paon, '') ||
                coalesce(saon, '') ||
                coalesce(street, '') ||
                coalesce(locality, '') ||
                coalesce(town, '') ||
                coalesce(district, '') ||
                coalesce(county, '') ||
                coalesce(postcode, '')
            as varbinary))) AS address_id,
                transaction_id,
                price,
                date_of_transfer,
                property_type,
                new_property,
                duration,
                ppd_category_type
        FROM latest_records
        WHERE record_status != 'D' and ppd_category_type = 'A'
    )
    SELECT *
    FROM with_address_id
    )
"""

In [4]:
with engine.begin() as conn:
    num_rows_dim_address = conn.execute(sa.text(dim_address_sql)).fetchone()[0]
    num_rows_fct_prices = conn.execute(sa.text(fct_prices_sql)).fetchone()[0]

print(f"Created dim_address with {num_rows_dim_address:,} rows")
print(f"Created fct_prices with {num_rows_fct_prices:,} rows")

Created dim_address with 8,618,471 rows
Created fct_prices with 8,966,172 rows


Now that the data is loaded, we can create a Pyiceberg reference to it

In [5]:
fct_house_prices_t = catalog.load_table("housing.fct_house_prices")

For a change of pace, let's use `polars` to write our profits calculation. Some things are easier to express in SQL and some are nice to be able to do in Polars. The choice is yours!

In [6]:
polars_result = (
    pl.scan_iceberg(fct_house_prices_t)
    .with_columns(
        pl.col("date_of_transfer").min().over(pl.col("address_id")).alias("first_day"),
        pl.col("date_of_transfer").max().over(pl.col("address_id")).alias("last_day"),
        pl.col("price")
        .sort_by("date_of_transfer")
        .first()
        .over(pl.col("address_id"))
        .alias("first_price"),
        pl.col("price")
        .sort_by("date_of_transfer")
        .last()
        .over(pl.col("address_id"))
        .alias("last_price"),
    )
    .with_columns(
        pl.col("last_day").sub(pl.col("first_day")).dt.total_days().alias("days_held"),
        pl.col("last_price").sub(pl.col("first_price")).alias("profit"),
    )
    .filter(pl.col("days_held") != 0)
    .select(
        pl.col("address_id"),
        pl.col("first_day"),
        pl.col("last_day"),
        pl.col("first_price"),
        pl.col("last_price"),
        pl.col("days_held"),
        pl.col("profit"),
    )
    .unique()
).collect()

polars_result

/usr/local/lib/python3.12/site-packages/pyiceberg/avro/decoder.py:185: UserWarning: Falling back to pure Python Avro decoder, missing Cython implementation
  warnings.warn("Falling back to pure Python Avro decoder, missing Cython implementation")


address_id,first_day,last_day,first_price,last_price,days_held,profit
str,date,date,i32,i32,i64,i32
"""E9E782749287DBA783B4BDDB22693067""",2018-09-07,2022-04-28,"140,000","146,500","1,329","6,500"
"""005293A47009A5E76AA1EC692D378745""",2016-07-18,2020-10-16,"150,000","159,950","1,551","9,950"
"""BD036A74A6AE2B3B4F654BB5B650D3D7""",2017-02-17,2025-07-18,"286,000","358,000","3,073","72,000"
"""61F5CC017147F74E7B99F5E0CCE8B343""",2022-01-31,2023-02-24,"273,000","320,000",389,"47,000"
"""1B5229A901356DD48DC5190E68F116FF""",2017-03-17,2022-03-18,"120,000","135,000","1,827","15,000"
…,…,…,…,…,…,…
"""6B78A2CE84052C808263191074A03E12""",2015-08-12,2023-04-28,"157,000","226,500","2,816","69,500"
"""ABCBD35037A40741358513225894F5E5""",2015-07-17,2024-09-13,"104,000","179,000","3,346","75,000"
"""86C70F67C4F183C4E4F521E865F08F49""",2018-06-29,2019-08-30,"79,000","95,000",427,"16,000"


Let's store the results in a table for future reference - since `polars` is arrow-based, we can use it to define the schema as well if we don't care as much about the details of the resulting schema

In [7]:
profits_t = catalog.create_table_if_not_exists("housing.profits", schema=polars_result.to_arrow().schema)

In [8]:
profits_t.overwrite(polars_result.to_arrow())

/usr/local/lib/python3.12/site-packages/pyiceberg/table/__init__.py:686: UserWarning: Delete operation did not match any records
  warnings.warn("Delete operation did not match any records")


To round out the selection of query engines, we can use `daft` to query our newly created table and calculate the mean profits for a given year

In [9]:
import daft

(
    daft.read_iceberg(profits_t)
    .groupby(daft.col("first_day").dt.year().alias("year"))
    .agg(daft.col("profit").mean())
    .sort(daft.col("year"))
    .collect(10)
)

yearInt32,profitFloat64
2015,74667.76502419915
2016,62078.20410008704
2017,59926.1708247446
2018,60692.83043605331
2019,66274.62430447362
2020,68234.58746926069
2021,55962.14272264139
2022,32401.928857311283
2023,40593.09405077681
2024,48607.20605143721
